<img style="float: center; width: 100%" src="https://raw.githubusercontent.com/andrejkk/TalksImgs/master/FrontSlideUpperBan.png">
<p style="margin-bottom:2cm;"></p>


# Robot za kognitivni trening z uporabniku prilagojeno komunikacijo

### Implementacija končnega avtomata (FSM) za adaptivno interakcijo človek-robot

Ime Priimek: Luka Rizman

Predmet: Uporabniku prilagojena komunikacija

Datum: December 2024


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## Kazalo


[Povzetek](#povzetek)

1. [Uvod](#uvod)
    1. [Opredelitev področja](#opredelitev-podrocja)
    2. [Predstavitev problema](#predstavitev-problema) 
2. [Teoretično ozadje in trenutno stanje](#teoreticno-ozadje)
    1. [Teoretični vidiki problema](#teoreticni-vidiki-problema)
    2. [Algoritmi](#algoritmi)
    3. [Merjenje uspešnosti](#merjenje-uspesnosti-resitve)
3. [Eksperimentalni del](#eksperimentalni-del)
    1. [Podatki](#podatki)
    2. [Arhitektura sistema](#arhitektura-sistema)
    3. [Demonstracija delovanja](#demonstracija-delovanja)
4. [Zaključek in razprava](#zakljucek-in-razprava)
5. [Doseženi učni izidi](#dosezeni-ucni-izidi)
6. [Literatura in viri](#literatura-in-viri)



<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## Povzetek <a class="anchor" id="povzetek"></a>

Projekt predstavlja implementacijo **robota za kognitivni trening**, ki uporablja **končni avtomat (FSM)** za prilagajanje komunikacije glede na uporabnikovo stanje.
</br>

**1. Izhodišča:** </br>
- **Področje:** Interakcija človek-robot (HRI) v kontekstu kognitivnega treninga starejših ali oseb s posebnimi potrebami.
- **Problem:** Kako robot prepozna uporabnikovo čustveno/kognitivno stanje in ustrezno prilagodi svojo komunikacijo?
- **Raziskovalno vprašanje:** Ali lahko na pravila temelječ končni avtomat učinkovito prilagaja robotsko komunikacijo v realnem času?

**2. Rešitev:** </br>
- **Končni avtomat (FSM)** s 5 stanji: Pozdrav → Razlaga → Vaja → Odmor → Zaključek
- **Pravila (Rule Engine)** na osnovi triggerjev (vedenjski signali uporabnika)
- **Klasifikacija sej** v referenčne scenarije (mirna, zmedena, stresna, kritična seja)
- **Evalvacijski sistem** za analizo učinkovitosti prehodov med stanji

**3. Ključni rezultati:** </br>
- Delujoča Flask spletna aplikacija z interaktivno simulacijo
- Sistem za beleženje sej in interakcij v SQLite bazi
- Metrke za evalvacijo: linearnost prehodov, učinkovitost, razmerje pozitivnih/negativnih interakcij

<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## 1. Uvod <a class="anchor" id="uvod"></a>

Socialni roboti postajajo vse bolj prisotni v zdravstvu, rehabilitaciji in skrbi za starejše. Ključni izziv pri njihovem razvoju je **prilagajanje komunikacije** glede na trenutno stanje uporabnika – ali je zmeden, pod stresom, pozitiven ali potrebuje odmor.

Ta projekt implementira **končni avtomat (Finite State Machine - FSM)** za upravljanje komunikacije robota med kognitivnim treningom. Robot na osnovi **vedenjskih signalov** (triggerjev) uporabnika izbira ustrezne odzive in prilagaja potek seje.

**Motivacija:**
- Starejši uporabniki potrebujejo prilagojeno komunikacijo
- Preveč intenzivna interakcija lahko povzroči stres
- Robot mora znati prepoznati znake frustracije in ponuditi odmor

**Cilji projekta:**
1. Implementirati FSM za vodenje interakcije človek-robot
2. Definirati pravila za izbiro robotskega odziva
3. Razviti evalvacijski sistem za analizo sej
4. Ustvariti interaktivno simulacijo za testiranje


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 1.1. Opredelitev področja <a class="anchor" id="opredelitev-podrocja"></a>

**Interakcija človek-robot (Human-Robot Interaction - HRI)** je interdisciplinarno področje, ki proučuje komunikacijo in sodelovanje med ljudmi in roboti. V kontekstu **kognitivnega treninga** se roboti uporabljajo za:

- **Rehabilitacijo** po možganski kapi ali poškodbah
- **Stimulacijo kognitivnih funkcij** pri starejših (preprečevanje demence)
- **Terapevtske intervencije** pri avtizmu in drugih razvojnih motnjah

**Ključne zahteve za komunikacijo:**
- Robot mora biti **nevsiljiv** – ne sme povzročati dodatnega stresa
- Komunikacija mora biti **razumljiva** – jasna in prilagojena uporabniku
- Robot mora biti **odziven** – pravočasno reagirati na uporabnikove signale
- Sistem mora omogočati **eskalacijo** – ob težavah poklicati pomoč

**Uporabniški kontekst:**
Tipični uporabniki so starejše osebe (65+) v domovih za ostarele ali rehabilitacijskih centrih, ki opravljajo kognitivne vaje pod vodstvom robota. Robot jih vodi skozi naloge, pri čemer mora prepoznati znake utrujenosti, zmede ali frustracije.


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 1.2. Predstavitev problema <a class="anchor" id="predstavitev-problema"></a>

**Problem:** Kako implementirati sistem za prilagajanje robotske komunikacije v realnem času?

**Formalna opredelitev:**

Imamo:
- Množico **stanj** S = {S0, S1, S2, S3, S4} (pozdrav, razlaga, vaja, odmor, zaključek)
- Množico **triggerjev** T = {t1, t2, ..., tn} (vedenjski signali uporabnika)
- Množico **intentov** I = {i1, i2, ..., im} (interpretirani nameni)
- Množico **robotskih odzivov** R = {r1, r2, ..., rk} (govorni akti)

**Cilj:** Najti preslikavo f: S × T → S' × R, ki:
1. Maksimizira uporabnikovo zadovoljstvo
2. Minimizira eskalacije (negativne interakcije)
3. Vodi sejo do uspešnega zaključka

**Pristop:**
Uporabimo **deterministični končni avtomat (DFA)** z razširjenim modelom, ki vključuje:
- Štetje eskalacij za avtomatski predlog zaključka
- Štetje uspešnih korakov za napredovanje
- Klasifikacijo sej v referenčne scenarije


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## 2. Teoretično ozadje in trenutno stanje <a class="anchor" id="teoreticno-ozadje-in-trenutno-stanje"></a>

Za razumevanje projekta je potrebno poznavanje naslednjih konceptov:

1. **Končni avtomati (FSM)** – formalni model za opis sistemov z diskretnimi stanji
2. **Govorni akti (Speech Acts)** – teorija komunikacije po Searlu
3. **Afektivno računalništvo** – prepoznavanje in interpretacija čustev

**Obstoječe rešitve v literaturi:**
- Roboti NAO in Pepper za terapijo avtizma (Scassellati et al., 2018)
- Socially Assistive Robotics (SAR) za rehabilitacijo (Matarić et al., 2007)
- Dialoge sistemi na osnovi FSM za glasovne asistente

**Naš prispevek:**
Integracija FSM s sistemom pravil za izbiro odzivov in evalvacijski sistem za analizo sej.


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 2.1. Teoretični vidiki problema <a class="anchor" id="teoreticni-vidiki-problema"></a>

#### Končni avtomat (Finite State Machine - FSM)

FSM je matematični model računanja definiran kot 5-terica (S, Σ, δ, s0, F):
- S – končna množica stanj
- Σ – abeceda (množica vhodnih simbolov)
- δ: S × Σ → S – prehodna funkcija
- s0 ∈ S – začetno stanje
- F ⊆ S – množica končnih stanj

V našem primeru:
- S = {S0_GREETING, S1_EXPLANATION, S2_EXERCISE, S3_BREAK, S4_FEEDBACK}
- Σ = množica intentov (Positive affect, Frustrated, Confused, ...)
- s0 = S0_GREETING
- F = {S4_FEEDBACK}

#### Govorni akti (Speech Acts)

Po teoriji J.R. Searla razlikujemo:
- **Asertivi** – izjave o svetu ("Danes je lep dan")
- **Direktivi** – navodila, vprašanja ("Ponovi za mano")
- **Komisivi** – obljube, zaveze ("Pomagal ti bom")
- **Ekspresivi** – izražanje čustev ("Bravo!")
- **Deklarativi** – spreminjanje stanja ("Seja je zaključena")

Robot uporablja kombinacije govornih aktov glede na situacijo:
- `Expressive + Commissive`: "Vidim, da si utrujen – naj narediva premor?"
- `Directive (clarify)`: "Ali naj ponovim navodila?"

#### Kategorizacija intentov

Intente delimo na:
- **Pozitivne:** Engaged, Positive Affect, Acknowledgment, Ready, Completion
- **Negativne:** Confusion, Stress, Frustration, Disengagement, Overload
- **Nevtralne:** Waiting, Hesitation, Attention Shift, Transition


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 2.2. Algoritmi <a class="anchor" id="algoritmi"></a>

#### Algoritem 1: Prehod med stanji FSM

```
ALGORITEM: FSM_UPDATE(current_state, intent)
VHOD: current_state ∈ S, intent ∈ I
IZHOD: next_state ∈ S

1. IF intent ∈ NEGATIVE_INTENTS:
      escalation_count += 1
   ELSE IF intent ∈ POSITIVE_INTENTS:
      positive_count += 1
      
2. SWITCH current_state:
   CASE S0_GREETING: 
      next_state = S1_EXPLANATION
   CASE S1_EXPLANATION:
      IF intent ∈ POSITIVE_INTENTS OR intent ∈ NEUTRAL:
         next_state = S2_EXERCISE
      ELSE:
         next_state = S1_EXPLANATION  // ostani, pojasni
   CASE S2_EXERCISE:
      IF intent ∈ NEGATIVE_INTENTS:
         next_state = S3_BREAK
      ELSE IF success_steps >= MAX_SUCCESS:
         next_state = S4_FEEDBACK
      ELSE:
         success_steps += 1
         next_state = S2_EXERCISE
   CASE S3_BREAK:
      IF intent ∈ POSITIVE_INTENTS:
         next_state = S2_EXERCISE
      ELSE:
         next_state = S3_BREAK  // ostani v odmoru
   CASE S4_FEEDBACK:
      next_state = S4_FEEDBACK  // končno stanje

3. IF total_escalations >= MAX_ESCALATIONS:
      suggest_end = TRUE
      
4. RETURN next_state
```

#### Algoritem 2: Izbira pravila (Rule Selection)

```
ALGORITEM: SELECT_RULE(trigger)
VHOD: trigger ∈ T
IZHOD: rule ∈ RULES ali NULL

1. matches = {r ∈ RULES | r.trigger == trigger}
2. IF matches = ∅:
      RETURN NULL
3. SORT matches BY priority DESC  // Critical > High > Medium > Low
4. RETURN matches[0]
```

#### Algoritem 3: Klasifikacija seje

```
ALGORITEM: CLASSIFY_SESSION(interactions)
VHOD: interactions - seznam interakcij seje
IZHOD: (scenario_id, confidence)

1. stats = CALCULATE_STATS(interactions)
   - positive_ratio, negative_ratio
   - total_escalations
   - final_state
   
2. FOR EACH scenario IN REFERENCE_SCENARIOS:
      score = 0
      IF stats.positive_ratio >= scenario.min_positive:
         score += 25
      IF stats.escalations <= scenario.max_escalations:
         score += 25
      IF stats.final_state == scenario.expected_final:
         score += 25
      IF stats.intents ∩ scenario.typical_intents ≠ ∅:
         score += 25
         
3. RETURN (best_scenario, best_score)
```

**Uporabljene knjižnice:**
- Flask 3.0 – spletni okvir
- Flask-SQLAlchemy 3.1 – ORM za SQLite bazo
- Python dataclasses – za definicijo FSM strukture


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 2.3. Merjenje uspešnosti rešitve <a class="anchor" id="merjenje-uspesnosti-resitve"></a>

Za evalvacijo sistema uporabljamo več metrik:

#### 1. Razmerje pozitivnih interakcij

**Positive Ratio** = |pozitivne interakcije| / |vse interakcije|

Ciljna vrednost: ≥ 0.6 (60% interakcij naj bo pozitivnih)

#### 2. Linearnost prehodov

**Linearity** = forward_moves / (forward_moves + backward_moves)

Meri, kako direkten je bil prehod skozi stanja. Vrednost 1.0 pomeni, da ni bilo povratkov v prejšnja stanja.

#### 3. Učinkovitost (Efficiency)

**Efficiency** = min(1.0, min_steps / actual_steps)

kjer je min_steps = 5 (minimalno potrebnih korakov za zaključek).

#### 4. Klasifikacijska zanesljivost

Ujemanje seje z referenčnim scenarijem (0-100%):
- ≥ 70%: Visoka zanesljivost
- 50-70%: Srednja zanesljivost  
- < 50%: Nizka zanesljivost

#### Referenčni scenariji

| Scenarij | Opis | Značilnosti |
|----------|------|-------------|
| **Mirna seja** 😊 | Uporabnik pozitiven in sodelujoč | positive_ratio ≥ 0.7, max 1 eskalacija |
| **Pogosto zmeden** 🤔 | Potrebuje več ponovitev | positive_ratio 0.3-0.6, max 3 eskalacije |
| **Odvrača pozornost** 👀 | Pogosto gleda drugam | Tipični intenti: Disengagement |
| **Pod stresom** 😰 | Znaki stresa in preobremenjenosti | positive_ratio ≤ 0.4, min 2 eskalaciji |
| **Kritična seja** 🚨 | Veliko eskalacij, predčasen zaključek | min 3 eskalacije |


</br>
<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## 3. Eksperimentalni del <a class="anchor" id="eksperimentalni-del"></a>

### Navodila za zagon aplikacije

**1. Namestitev odvisnosti:**
```bash
pip install -r requirements.txt
```

**2. Zagon aplikacije:**
```bash
python app.py
```

**3. Dostop do vmesnika:**
- Glavna stran: http://localhost:5000/
- Evalvacija sej: http://localhost:5000/evaluate

**Potrebne knjižnice (requirements.txt):**
```
Flask==3.0.0
Flask-SQLAlchemy==3.1.1
pymysql==1.1.0
python-dotenv==1.0.1
```

**Struktura projekta:**
```
RobotKoncniAvtomat/
├── app.py              # Glavna Flask aplikacija
├── config.py           # Konfiguracija
├── core/               # Jedro sistema
│   ├── fsm.py          # Končni avtomat
│   └── rules_loader.py # Pravila
├── db/                 # Baza podatkov
│   └── models.py       # SQLAlchemy modeli
├── evaluation/         # Evalvacijski sistem
│   ├── functions.py    # Evalvacijske funkcije
│   ├── categories.py   # Kategorije intentov
│   └── scenarios.py    # Referenčni scenariji
├── routes/             # API endpoints
├── templates/          # HTML predloge
└── static/             # CSS, JS
```


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 3.1. Podatki <a class="anchor" id="podatki"></a>

#### Pravila (Rules)

Sistem uporablja 14 pravil definiranih v `core/rules_loader.py`. Vsako pravilo vsebuje:

| Polje | Opis | Primer |
|-------|------|--------|
| **Trigger** | Vedenjski signal uporabnika | "User face stressed" |
| **Inferred Intent** | Interpretiran namen | "User frustrated / overloaded" |
| **Priority** | Prioriteta (Critical/High/Medium/Low) | "Medium" |
| **ConfidenceThrs.** | Prag zaupanja | 0.60 |
| **Escalated Action** | Eskalacijska akcija | "Call for help" |
| **Robot Speech Act** | Tip govornega akta | "Expressive + Commissive" |
| **RobotText** | Robotov odziv | "You look stressed — shall I slow down?" |

#### Primeri pravil:

```python
{
    "Trigger": "User face stressed",
    "Inferred Intent": "User frustrated / overloaded",
    "Priority": "Medium",
    "ConfidenceThrs.": 0.60,
    "Robot Speech Act": "Expressive + Commissive",
    "RobotText": "You look stressed — shall I slow down?"
},
{
    "Trigger": "User smiles/laughs",
    "Inferred Intent": "Positive affect",
    "Priority": "Medium",
    "Robot Speech Act": "Expressive",
    "RobotText": "Haha, that's funny!"
}
```

#### Baza podatkov

SQLite baza (`robot_fsm.db`) vsebuje dve tabeli:

**Tabela `sessions`:**
- id, started_at, ended_at
- rating_supportive, rating_understandable, rating_non_intrusive (1-5 Likert)
- evaluated_at

**Tabela `interactions`:**
- id, session_id, step_number, timestamp
- state_before, state_after
- trigger, inferred_intent
- robot_speech_act, robot_utterance
- priority, escalation_count


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 3.2. Arhitektura sistema <a class="anchor" id="arhitektura-sistema"></a>

#### Diagram stanj FSM

```
                    ┌──────────────┐
                    │ S0_GREETING  │
                    │   👋 Pozdrav │
                    └──────┬───────┘
                           │ vedno
                           ▼
                    ┌──────────────────┐
            ┌───────│ S1_EXPLANATION   │◄──────┐
            │       │   📖 Razlaga     │       │
            │       └────────┬─────────┘       │
            │                │ pozitiven/      │ negativen
            │                │ nevtralen       │
            │                ▼                 │
            │       ┌──────────────────┐       │
            │   ┌──►│   S2_EXERCISE    │───────┘
            │   │   │    🎯 Vaja       │
            │   │   └────────┬─────────┘
            │   │            │
            │   │  pozitiven │ negativen
            │   │            ▼
            │   │   ┌──────────────────┐
            │   └───│    S3_BREAK      │
            │       │    ☕ Odmor      │
            │       └────────┬─────────┘
            │                │ feedback / 
            │                │ success_steps >= 5
            │                ▼
            │       ┌──────────────────┐
            └──────►│   S4_FEEDBACK    │
                    │   ✅ Zaključek   │
                    └──────────────────┘
```

#### Arhitektura aplikacije

```
┌─────────────────────────────────────────────────────────┐
│                    FLASK APP                            │
├─────────────────────────────────────────────────────────┤
│  ┌─────────────┐   ┌─────────────┐   ┌─────────────┐   │
│  │   routes/   │   │    core/    │   │    db/      │   │
│  │  main.py    │──►│   fsm.py    │   │  models.py  │   │
│  │ evaluate.py │   │rules_loader │   │             │   │
│  └─────────────┘   └─────────────┘   └─────────────┘   │
│         │                │                  │          │
│         │                │                  │          │
│         ▼                ▼                  ▼          │
│  ┌─────────────────────────────────────────────────┐   │
│  │              evaluation/                        │   │
│  │  functions.py │ categories.py │ scenarios.py   │   │
│  └─────────────────────────────────────────────────┘   │
├─────────────────────────────────────────────────────────┤
│  templates/        static/css/         static/js/      │
│  index.html        main.css            main.js         │
│  evaluate.html     evaluate.css        evaluate.js     │
└─────────────────────────────────────────────────────────┘
```

#### API Endpoints

| Metoda | Endpoint | Opis |
|--------|----------|------|
| GET | `/` | Glavna stran simulacije |
| POST | `/trigger` | Obdelava triggerja |
| POST | `/reset` | Ponastavitev seje |
| POST | `/force-end` | Prisili zaključek |
| GET | `/statistics` | Statistika trenutne seje |
| GET | `/evaluate` | Stran za evalvacijo |
| GET | `/api/all-sessions` | Seznam vseh sej |
| GET | `/api/session/<id>` | Podrobnosti seje |


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

### 3.3. Demonstracija delovanja <a class="anchor" id="demonstracija-delovanja"></a>

#### Primer mirne seje

**Potek:**
1. `greet` → S0→S1 → "Hello! Nice to see you!"
2. `User smiles/laughs` → S1→S2 → "Haha, that's funny!"
3. `User smiles/laughs` → S2→S2 → "Haha, that's funny!"
4. `end of user speech` → S2→S2 → "I understand. Let me continue..."
5. `User smiles/laughs` → S2→S4 (5 uspešnih korakov) → zaključek

**Rezultat evalvacije:**
- Scenarij: "Mirna seja" 😊
- Confidence: 85%
- Positive ratio: 80%
- Linearnost: 100%

#### Primer stresne seje

**Potek:**
1. `greet` → S0→S1 → "Hello! Nice to see you!"
2. `User face stressed` → S1→S1 → "You look stressed — shall I slow down?"
3. `User leans back / crosses arms` → S1→S2→S3 → "Should we take a short break?"
4. `User face stressed` → S3→S3 → "You look stressed — shall I slow down?"
5. (3 eskalacije) → robot predlaga zaključek

**Rezultat evalvacije:**
- Scenarij: "Pod stresom" 😰
- Confidence: 78%
- Positive ratio: 20%
- Eskalacije: 3

#### Statistika seje (primer)

```json
{
  "step_count": 8,
  "positive_interactions": 5,
  "negative_interactions": 2,
  "total_escalations": 2,
  "success_steps": 4,
  "current_state": "S2_EXERCISE",
  "is_final": false,
  "positive_ratio": 0.71,
  "should_suggest_end": false
}
```


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## 4. Zaključek in razprava <a class="anchor" id="zakljucek-in-razprava"></a>

### Glavni rezultati

1. **Uspešno implementiran FSM** za vodenje interakcije človek-robot s 5 stanji in logiko prehodov
2. **Rule Engine** z 14 pravili za izbiro ustreznega robotskega odziva
3. **Evalvacijski sistem** za klasifikacijo sej v 5 referenčnih scenarijev
4. **Interaktivna spletna aplikacija** za testiranje in demonstracijo

### Zaključki

- FSM pristop je **primeren** za modeliranje dialogov v HRI
- Sistem pravil omogoča **prilagodljivost** in enostavno razširjanje
- Evalvacijske metrike dajejo **uporabne povratne informacije** o kvaliteti sej
- Klasifikacija scenarijev omogoča **primerjavo** med različnimi sejami

### Omejitve

- Sistem ne vključuje **dejanskega prepoznavanja** vedenjskih signalov (samo simulacija)
- Pravila so **statična** – ni strojnega učenja za izboljšanje odzivov
- **Ni testiranja z realnimi uporabniki** – potrebna bi bila uporabniška študija

### Prihodnje delo

1. **Integracija z računalniškim vidom** za detekcijo obrazne mimike
2. **Reinforcement learning** za optimizacijo prehodov FSM
3. **Večjezična podpora** (slovenščina, angleščina)
4. **Uporabniška študija** s ciljno populacijo
5. **Integracija z robotom NAO/Pepper** za realno testiranje


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## 5. Doseženi učni izidi <a class="anchor" id="dosezeni-ucni-izidi"></a>

Pri izdelavi projekta sem se naučil:

### Razumevanje teoretičnih konceptov
- **Končni avtomati (FSM)** – kako modelirati sisteme z diskretnimi stanji
- **Govorni akti** – teorija komunikacije in vrste izjav
- **Interakcija človek-robot (HRI)** – principi prilagajanja komunikacije

### Spoznavanje algoritmov
- Implementacija **deterministični končnega avtomata** v Pythonu
- **Rule-based sistemi** za izbiro odzivov na osnovi prioritet
- **Evalvacijske metrike** za analizo interakcij

### Pridobljene izkušnje s podatki in tehnologijami
- **Flask** spletni okvir za Python
- **SQLAlchemy** ORM za delo z bazami podatkov
- **Strukturiranje projekta** v module (core, db, evaluation, routes)
- **REST API** načrtovanje za spletne aplikacije

### Praktične veščine
- Načrtovanje **uporabniškega vmesnika** za interaktivno simulacijo
- **Beleženje in analiza** podatkov o sejah
- Pisanje **dokumentacije** za tehnični projekt


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">

## 6. Literatura in viri  <a class="anchor" id="literatura-in-viri"></a>

### Znanstvena literatura

1. **Searle, J. R. (1969).** *Speech Acts: An Essay in the Philosophy of Language.* Cambridge University Press.
   - Osnovna teorija govornih aktov.

2. **Scassellati, B., Admoni, H., & Mataric, M. (2012).** Robots for use in autism research. *Annual Review of Biomedical Engineering*, 14, 275-294.
   - Pregled uporabe robotov v terapiji avtizma.

3. **Mataric, M. J., & Scassellati, B. (2016).** Socially assistive robotics. In *Springer Handbook of Robotics* (pp. 1973-1994).
   - Pregled področja socialno asistivne robotike.

4. **Tapus, A., Tapus, C., & Mataric, M. J. (2008).** User—robot personality matching and assistive robot behavior adaptation for post-stroke rehabilitation therapy. *Intelligent Service Robotics*, 1(2), 169-183.
   - Prilagajanje robotskega vedenja v rehabilitaciji.

5. **Feil-Seifer, D., & Mataric, M. J. (2005).** Defining socially assistive robotics. *IEEE ICORR 2005*, 465-468.
   - Definicija socialno asistivne robotike.

### Tehnicna dokumentacija

6. **Flask Documentation.** https://flask.palletsprojects.com/
   - Uradna dokumentacija Flask okvira.

7. **SQLAlchemy Documentation.** https://docs.sqlalchemy.org/
   - Dokumentacija SQLAlchemy ORM.

8. **Python dataclasses.** https://docs.python.org/3/library/dataclasses.html
   - Dokumentacija Python dataclasses modula.

### Dodatni viri

9. **Hopcroft, J. E., Motwani, R., & Ullman, J. D. (2006).** *Introduction to Automata Theory, Languages, and Computation.* Pearson.
   - Teoreticne osnove koncnih avtomatov.

10. **Breazeal, C. (2003).** Toward sociable robots. *Robotics and Autonomous Systems*, 42(3-4), 167-175.
    - Principi razvoja socialnih robotov.


<p style="margin-bottom:2cm;"></p>
<img src="https://raw.githubusercontent.com/andrejkk/ORvTK_SlidesImgs/master/footer_full.jpg">